In [2]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import SGD
import cv2
import skimage
import os
from imgaug.imgaug import augmenters as iaa
from densenet121 import DenseNet
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from random import shuffle

Using TensorFlow backend.


In [38]:
class biz_util:
    def __init__ (self, data_dir, biz_label_file_name, photo_biz_file_name):
        self.batch_index = 0
        self.image_paths = [os.path.join(data_dir,i) for i in os.listdir(data_dir) if i.endswith('.jpg') and not i.startswith("._")]
        self.image_len = len(self.image_paths)
        self.biz_label = self.biz_id_to_label(biz_label_file_name)
        self.biz_photo = self.biz_id_to_photo(photo_biz_file_name)
        self.data_dir = data_dir
        
    def next_batch(self, batch_size):
        images = []
        labels = []
        label_photos = {}
        # 2000 biz in total
        if batch_size + self.batch_index * batch_size < len(self.biz_photo):
            start = self.batch_index * batch_size
            end = start + batch_size
            self.batch_index += 1
        # for last batch
        else:
            start = self.batch_index * batch_size
            end = len(self.biz_photo)
            self.batch_index = 0
            
        for biz in self.shuffle_biz_photo_keys[start:end]:
            label = tuple(self.biz_label[biz])
            labels_for_biz = []
            # for each biz, get 100 photos for training
            photo_ids = self.biz_photo[biz]
            none = 0
            for img_id in photo_ids[:]:
                img = cv2.imread(os.path.join(self.data_dir, img_id+'.jpg'))
                if img is None:
                    none += 1
                    continue
                #img resize & normalization
                img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
                img[:,:,0] = (img[:,:,0] - 103.94) * 0.017
                img[:,:,1] = (img[:,:,1] - 116.78) * 0.017
                img[:,:,2] = (img[:,:,2] - 123.68) * 0.017
                
                labels_for_biz.append(img)
#             print none
#             print(len(label_photos[label]))
            if labels_for_biz:
                label_photos[label] = np.asarray(labels_for_biz)
        labels = np.asarray(label_photos.keys())
        images = np.asarray(label_photos.values())
        
        return images, labels
    
    def shuffle_biz(self):
        self.shuffle_biz_photo_keys = self.biz_photo.keys()
        shuffle(self.shuffle_biz_photo_keys)
        
#     # business features
#     def get_biz_features(self, biz_label, biz_photo):
#         biz_imgs = {}
#         imgs = []
#         for biz in biz_photo:
#             label = tuple(self.biz_label[biz])
#             # for each biz, get 100 photos for training
#             photo_ids = self.biz_photo[biz]
#             none = 0
#             for img_id in photo_ids[:]:
#                 img = cv2.imread(os.path.join(self.data_dir, img_id+'.jpg'))
#                 if img is None:
#                     none += 1
#                     continue
#                 #img resize & normalization
#                 img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
#                 img[:,:,0] = (img[:,:,0] - 103.94) * 0.017
#                 img[:,:,1] = (img[:,:,1] - 116.78) * 0.017
#                 img[:,:,2] = (img[:,:,2] - 123.68) * 0.017
#                 imgs.append(img)
#             if imgs:
#                 biz_imgs[biz] = np.asarray(imgs)
#         biz_ids = np.asarray(biz_imgs.keys())
#         images = np.asarray(biz_imgs.values())
        
#         for biz_id, img_list in biz_imgs:
#             pred = new_model.predict(img_list)
#     #         print('hiuhiuhiu')
#             biz_imgs[biz_id] = np.mean(pred, axis=0)
            
#         return biz_imgs
                
    ''' get dictionary of biz_ids and all the corresponding photos '''
    def biz_id_to_photo(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        biz_to_photo = {}
        for line in lines:
            if line.split(",")[1].rstrip() not in biz_to_photo.keys():
                biz_to_photo[line.split(",")[1].rstrip()] = [line.split(",")[0]]
            else:
                biz_to_photo[line.split(",")[1].rstrip()].append(line.split(",")[0])
        return biz_to_photo
    
    ''' get labels for each biz_id, representing in a multi-hot vector '''
    def biz_id_to_label(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        biz_id_to_label = {}
        for line in lines:
            try:
                biz_id_to_label[line.split(",")[0]] = np.zeros(9)
                for label in line.split(",")[1].rstrip().split(' '):
                    biz_id_to_label[line.split(",")[0]][int(label)]=1
            except:
                if not line.split(",")[1].rstrip():
                    continue
        return biz_id_to_label

In [3]:
model = DenseNet(reduction=0.5, classes=1000, weights_path='./densenet121_weights_tf.h5')
print(model.layers[-1].output_shape)
model.layers.pop()
model.layers.pop()
output = model.layers[-1].output
new_model = Model(model.input, output)

densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:164: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/home/wanlu/4032/venv/local/lib/pyt

densenet121.py:135: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), use_bias=False, name="conv3_blk")`
  x = Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_2_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="co

densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_22_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_22_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_23_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv4_23_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv4_24_x1

(None, 1000)


In [4]:
new_model.layers[-1].output_shape

(None, 1024)

In [7]:
# business features
def get_biz_features(images):
    biz_features = []
    for i, img_list in enumerate(images):
        pred = new_model.predict(img_list)
#         print('hiuhiuhiu')
        biz_features.append(np.mean(pred, axis=0))
    return np.asarray(biz_features)

In [34]:
def get_img_features(img_list):
    pred = new_model.predict(img_list)
    return np.asarray(pred)

In [36]:
inputs = biz_util('../data/train_photos', './train.csv', './train_photo_to_biz_ids.csv')
# inputs.next_batch(8)[0].shape

In [5]:
from sklearn.utils import shuffle

In [ ]:
# brave new world

In [6]:
train_features = np.load('../data/features_234000.npy')
train_labels = np.load('../data/features_labels_234000.npy')

In [7]:
def get_data(train_features, train_labels):
#     train_features, train_labels = shuffle(train_features, train_labels, random_state=7)
    
    label_to_features = {}
    for index, label in enumerate(train_labels):
        label = tuple(label)
        if label not in label_to_features:
            label_to_features[label] = []
        else:
            label_to_features[label].append(train_features[index])
    labels = []
    img_features = []
    for l in label_to_features.keys():
        split = len(label_to_features[l])/100 + 1
        for i in range(split):
            labels.append(l)
            if i == split-1:
                img_features.append(np.asarray(label_to_features[l][i*100:]))
            else:
                img_features.append(np.asarray(label_to_features[l][i*100:(i+1)*100]))
    labels = np.asarray(labels)
    img_features = np.asarray(img_features)
    biz_features = np.asarray([np.mean(f, axis=0) for f in img_features])
    
    biz_features, img_features, labels = shuffle(biz_features, img_features, labels, random_state=7)
    
    return biz_features, img_features, labels

In [8]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
def one_vs_rest_train_test(train_X, train_y, test_X, test_y):
    
    # classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True))
    classifier = OneVsRestClassifier(LinearSVC(loss='hinge', random_state=0))
    classifier.fit(train_X, train_y)
    
    from sklearn.metrics import f1_score
    print "test"
    y_predict = classifier.predict(test_X)
    print "F1 score: ", f1_score(test_y, y_predict, average='micro')
    print "Individual Class F1 score: ", f1_score(test_y, y_predict, average=None)

In [127]:
biz_features, img_features, labels = get_data(train_features, train_labels)
print biz_features.shape
print img_features.shape
print labels.shape

(2402, 1024)
(2402,)
(2402, 9)


In [12]:
epoch = 5
display_step = 20
lst_clfs = [LinearSVC() for i in range(9)]
for i in range(epoch):
    biz_features, img_features, labels = get_data(train_features, train_labels)
    train_X = biz_features[:2000]
    train_y = labels[:2000]
    val_X = img_features[2000:]
    val_y = labels[2000:]
#     one_vs_rest_train_test(train_X, train_y, val_X, val_y)
    
    for k in range(9):
        clf = lst_clfs[k]
        clf.fit(train_X, train_y[:, k])
    # test
    vimages = val_X[0]
    for ims in val_X[1:]:
        vimages = np.vstack((vimages,ims))
    vlabels = np.repeat(val_y, [im.shape[0] for im in val_X], axis=0)
    accs = []
    for k in range(9):
        clf = lst_clfs[k]
        accs.append(round(clf.score(vimages, vlabels[:, k]), 3))
    print('Epoch:'+str(i)+'  '+'Accuracy:'+str(accs))

Epoch:0  Accuracy:[0.712, 0.711, 0.703, 0.681, 0.719, 0.725, 0.733, 0.687, 0.683]
Epoch:1  Accuracy:[0.716, 0.711, 0.698, 0.68, 0.715, 0.721, 0.732, 0.685, 0.683]
Epoch:2  Accuracy:[0.707, 0.714, 0.704, 0.68, 0.713, 0.725, 0.733, 0.684, 0.684]
Epoch:3  Accuracy:[0.716, 0.713, 0.702, 0.679, 0.719, 0.724, 0.733, 0.686, 0.685]
Epoch:4  Accuracy:[0.718, 0.713, 0.701, 0.683, 0.718, 0.725, 0.732, 0.683, 0.688]
